众所周知，在Python中如果访问字典中不存在的键，会引发KeyError异常（JavaScript中如果对象中不存在某个属性，则返回undefined）。但是有时候，字典中的每个键都存在默认值是非常方便的。例如下面的例子：

In [5]:
strings = ('puppy', 'kitten', 'puppy', 'puppy',
           'weasel', 'puppy', 'kitten', 'puppy')
counts = {}

for kw in strings:
    counts[kw] += 1

KeyError: 'puppy'

该例子统计strings中某个单词出现的次数，并在counts字典中作记录。单词每出现一次，在counts相对应的键所存的值数字加1。但是事实上，运行这段代码会抛出KeyError异常，出现的时机是每个单词第一次统计的时候，因为Python的dict中不存在默认值的说法，可以在Python命令行中验证：

In [6]:
counts = dict()
counts['puppy'] += 1

KeyError: 'puppy'

# 使用判断语句检查
既然如此，首先可能想到的方法是在单词第一次统计的时候，在counts中相应的键存下默认值1。这需要在处理的时候添加一个判断语句：

In [8]:
strings = ('puppy', 'kitten', 'puppy', 'puppy',
           'weasel', 'puppy', 'kitten', 'puppy')
counts = {}

for kw in strings:
    if kw not in counts:
        counts[kw] = 1
    else:
        counts[kw] += 1

counts

{'puppy': 5, 'kitten': 2, 'weasel': 1}

# 使用dict.setdefault()方法
也可以通过dict.setdefault()方法来设置默认值：

In [9]:
strings = ('puppy', 'kitten', 'puppy', 'puppy',
           'weasel', 'puppy', 'kitten', 'puppy')
counts = {}

for kw in strings:
    counts.setdefault(kw, 0)
    counts[kw] += 1
counts

{'puppy': 5, 'kitten': 2, 'weasel': 1}

dict.setdefault()方法接收两个参数，第一个参数是健的名称，第二个参数是默认值。假如字典中不存在给定的键，则返回参数中提供的默认值；反之，则返回字典中保存的值。利用dict.setdefault()方法的返回值可以重写for循环中的代码，使其更加简洁：

In [10]:
strings = ('puppy', 'kitten', 'puppy', 'puppy',
           'weasel', 'puppy', 'kitten', 'puppy')
counts = {}

for kw in strings:
    counts[kw] = counts.setdefault(kw, 0) + 1
counts

{'puppy': 5, 'kitten': 2, 'weasel': 1}

# 使用collections.defaultdict类
以上的方法虽然在一定程度上解决了dict中不存在默认值的问题，但是这时候我们会想，有没有一种字典它本身提供了默认值的功能呢？答案是肯定的，那就是collections.defaultdict。

defaultdict类就好像是一个dict，但是它是使用一个类型来初始化的：

In [11]:
from collections import defaultdict
dd = defaultdict(list)
dd

defaultdict(list, {})

defaultdict类的初始化函数接受一个类型作为参数，当所访问的键不存在的时候，可以实例化一个值作为默认值：

In [12]:
dd['foo']

[]

In [13]:
dd

defaultdict(list, {'foo': []})

In [14]:
dd['bar'].append('quux')

In [15]:
dd

defaultdict(list, {'foo': [], 'bar': ['quux']})

需要注意的是，这种形式的默认值只有在通过dict[key]或者dict.__getitem__(key)访问的时候才有效，这其中的原因在下文会介绍。

In [16]:
from collections import defaultdict
dd = defaultdict(list)
dd

defaultdict(list, {})

In [17]:
'something' in dd

False

In [18]:
dd.pop('something')

KeyError: 'something'

In [19]:
dd.get('something')

In [21]:
dd['something']

[]

defaultdict类除了接受类型名称作为初始化函数的参数之外，还可以使用任何不带参数的可调用函数，到时该函数的返回结果作为默认值，这样使得默认值的取值更加灵活。下面用一个例子来说明，如何用自定义的不带参数的函数zero()作为defaultdict类的初始化函数的参数：

In [22]:
from collections import defaultdict
def zero():
    return 0

dd = defaultdict(zero)
dd

defaultdict(<function __main__.zero()>, {})

In [23]:
dd['foo']

0

In [24]:
dd

defaultdict(<function __main__.zero()>, {'foo': 0})

利用collections.defaultdict来解决最初的单词统计问题，代码如下：

In [25]:
from collections import defaultdict

strings = ('puppy', 'kitten', 'puppy', 'puppy',
           'weasel', 'puppy', 'kitten', 'puppy')
counts = defaultdict(lambda: 0)  # 使用lambda来定义简单的函数

for s in strings:
    counts[s] += 1

counts

defaultdict(<function __main__.<lambda>()>,
            {'puppy': 5, 'kitten': 2, 'weasel': 1})

# defaultdict类是如何实现的
通过上面的内容，想必大家已经了解了defaultdict类的用法，那么在defaultdict类中又是如何来实现默认值的功能呢？这其中的关键是使用了看__missing__()这个方法：

In [28]:
from collections import defaultdict
print(defaultdict.__missing__.__doc__)

__missing__(key) # Called by __getitem__ for missing key; pseudo-code:
  if self.default_factory is None: raise KeyError((key,))
  self[key] = value = self.default_factory()
  return value



通过查看__missing__()方法的docstring，可以看出当使用__getitem__()方法访问一个不存在的键时(dict[key]这种形式实际上是__getitem__()方法的简化形式)，会调用__missing__()方法获取默认值，并将该键添加到字典中去。

关于__missing__()方法的具体介绍可以参考Python官方文档中的"[Mapping Types — dict](http://docs.python.org/library/stdtypes.html#dict)"一节。

文档中介绍，从2.5版本开始，如果派生自dict的子类定义了__missing__()方法，当访问不存在的键时，dict[key]会调用__missing__()方法取得默认值。

从中可以看出，虽然dict支持__missing__()方法，但是在dict本身是不存在这个方法的，而是需要在派生的子类中自行实现这个方法。可以简单的验证这一点：

In [29]:
print(dict.__missing__.__doc__)

AttributeError: type object 'dict' has no attribute '__missing__'

同时，我们可以进一步的做实验，定义一个子类Missing并实现__missing__()方法:

In [30]:
class Missing(dict):
    def __missing__(self, key):
        return 'missing'

In [33]:
d = Missing()
d

{}

In [34]:
d['foo']

'missing'

In [35]:
d

{}

返回结果反映了__missing__()方法确实发挥了作用。在此基础上，我们稍许修改__missing__()方法,使得该子类同defautldict类一样为不存在的键设置一个默认值：

In [44]:
class Defaulting(dict):
    def __missing__(self, key):
        print('__missing__', key)
        self[key] = 'default'
        return 'default'

In [45]:
d = Defaulting()
d

{}

In [46]:
d['foo']

__missing__ foo


'default'

In [47]:
d

{'foo': 'default'}

In [48]:
d['foo']

'default'

In [49]:
help(defaultdict)

Help on class defaultdict in module collections:

class defaultdict(builtins.dict)
 |  defaultdict(default_factory=None, /, [...]) --> dict with default factory
 |  
 |  The default factory is called without arguments to produce
 |  a new value when a key is not present, in __getitem__ only.
 |  A defaultdict compares equal to a dict with the same items.
 |  All remaining arguments are treated the same as if they were
 |  passed to the dict constructor, including keyword arguments.
 |  
 |  Method resolution order:
 |      defaultdict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(...)
 |      D.copy() -> a shallow copy of D.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __missing__(...)
 |      __missing__(key) # Called by __getitem__ for missing key; pseudo-code:
 |      if self.defau